In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
df = pd.read_csv("sales_pos.csv")
df.head(2)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200


### Q1. 

In [ ]:
# df.groupby("prod")["purchase"].sum()
df.groupby("prod")["purchase"].sum().max()

In [ ]:
df_prod_sum = df.groupby("prod")["purchase"].sum()
df_prod_sum[df_prod_sum == df_prod_sum.max()]

In [ ]:
top_prod = df.groupby("prod")["purchase"].sum().idxmax()
top_prod

In [9]:
df_q1 = df.loc[df["prod"] == top_prod, ["prod", "job"]]
df_q1.head(2)

,prod,job
667,P00025442,17
749,P00025442,7


In [11]:
df_q1["job"].value_counts().idxmax()

4

### Q2.

In [14]:
df_u1 = df.loc[df["user"] == 1, ["prod_cat1", "prod_cat2", "prod_cat3"]].reset_index(drop = True)
df_u1 = df_u1.fillna(0)
df_u1.head(2)

,prod_cat1,prod_cat2,prod_cat3
0,3,0.0,0.0
1,1,6.0,14.0


In [15]:
df_u1.sort_values(["prod_cat1", "prod_cat2", "prod_cat3"])

,prod_cat1,prod_cat2,prod_cat3
6,1,2.0,5.0
29,1,2.0,9.0
1,1,6.0,14.0
15,1,8.0,17.0
4,2,4.0,8.0
0,3,0.0,0.0
22,3,4.0,0.0
24,3,4.0,0.0
25,3,4.0,0.0
11,3,4.0,5.0


In [21]:
len(df_u1)

In [20]:
len(df_u1.drop_duplicates())

In [18]:
# 정수로 굳이 바꾸지 않아도 되나.... 이쁜 출력을 위해...ㅎ...
df_u1["prod_cat1"] = df_u1["prod_cat1"].astype("int").astype("str")
df_u1["prod_cat2"] = df_u1["prod_cat2"].astype("int").astype("str")
df_u1["prod_cat3"] = df_u1["prod_cat3"].astype("int").astype("str")
df_u1["prod_cat" ] = df_u1["prod_cat1"] + "-" + df_u1["prod_cat2"] + "-" + df_u1["prod_cat3"]

In [22]:
df_u1.head(2)

In [23]:
df_u1["prod_cat"].nunique()

21

In [26]:
df_q2 = df[["user", "marital", "age_group", "prod_cat1", "prod_cat2", "prod_cat3"]]
df_q2 = df_q2.loc[df_q2["age_group"] == "26-35", ].reset_index(drop = True)
df_q2 = df_q2.fillna(0)
df_q2.head(2)

,user,marital,age_group,prod_cat1,prod_cat2,prod_cat3
0,3,0,26-35,1,2.0,0.0
1,5,1,26-35,8,0.0,0.0


In [27]:
df_q2["prod_cat1"] = df_q2["prod_cat1"].astype("int").astype("str")
df_q2["prod_cat2"] = df_q2["prod_cat2"].astype("int").astype("str")
df_q2["prod_cat3"] = df_q2["prod_cat3"].astype("int").astype("str")
df_q2["prod_cat" ] = df_q2["prod_cat1"] + "-" + df_q2["prod_cat2"] + "-" + df_q2["prod_cat3"]

In [28]:
df_q2.head(2)

,user,marital,age_group,prod_cat1,prod_cat2,prod_cat3,prod_cat
0,3,0,26-35,1,2,0,1-2-0
1,5,1,26-35,8,0,0,8-0-0


In [33]:
df_q2_cnt = df_q2.groupby(["user", "marital"])["prod_cat"].nunique().reset_index()
df_q2_cnt.head()

In [34]:
stat_m0 = df_q2_cnt.loc[df_q2_cnt["marital"] == 0, "prod_cat"].mean()
stat_m1 = df_q2_cnt.loc[df_q2_cnt["marital"] == 1, "prod_cat"].mean()
round(abs(stat_m0 - stat_m1), 2)

0.13

### Q3.

In [ ]:
df_user = df[["user", "gender", "age_group", "job", "city", "marital"]].drop_duplicates()
df_user.head(2)

In [ ]:
df_user_s1 = df_user.head(1)
df_user_s1["city"] = "D"
df_user_s1

In [43]:
df_bind = pd.concat([df_user_s1, df_user]).reset_index(drop = True)
df_bind.head()

,user,gender,age_group,job,city,marital
0,1,F,0-17,10,D,0
1,1,F,0-17,10,A,0
2,2,M,55+,16,C,0
3,3,M,26-35,15,A,0
4,4,M,46-50,7,B,1


In [ ]:
ser_u_cnt = df_bind.groupby("user")["city"].nunique()
ser_u_cnt[ser_u_cnt >= 2] # 이사를 한 것으로 추정되는 고객

In [ ]:
len(df_user), df["user"].nunique()

In [47]:
df_agg = df.groupby("user")[["prod", "purchase"]].agg({"prod": "nunique", 
                                                       "purchase": "sum"}).reset_index()
df_agg.head(2)

,user,prod,purchase
0,1,35,334093
1,2,77,810472


In [48]:
col_groupby = ["user", "gender", "age_group", "job", "city", "marital"]
df_join = df.groupby(col_groupby)[["prod", "purchase"]].agg({"prod": "nunique", 
                                                             "purchase": "sum"}).reset_index()
df_join.head(3)

,user,gender,age_group,job,city,marital,prod,purchase
0,1,F,0-17,10,A,0,35,334093
1,2,M,55+,16,C,0,77,810472
2,3,M,26-35,15,A,0,29,341635


In [49]:
df_join["gender"] = df_join["gender"].replace({"M": 1, "F": 0})

In [51]:
arr_u = df_join["age_group"].unique()
# arr_u = arr_u.sort() # <-- 이렇게 쓰지 않음...
arr_u.sort() # array라 문법이 조금 다름..
arr_u

array(['0-17', '18-25', '26-35', '36-45', '46-50', '51-55', '55+'],
      dtype=object)

In [52]:
ser_repl = pd.Series(range(7), index = arr_u)
ser_repl

0-17     0
18-25    1
26-35    2
36-45    3
46-50    4
51-55    5
55+      6
dtype: int64

In [53]:
pd.crosstab(df_join["age_group"], 
            df_join["age_group"].replace(ser_repl))

age_group,0,1,2,3,4,5,6
age_group,,,,,,,
0-17,218,0,0,0,0,0,0
18-25,0,1069,0,0,0,0,0
26-35,0,0,2053,0,0,0,0
36-45,0,0,0,1167,0,0,0
46-50,0,0,0,0,531,0,0
51-55,0,0,0,0,0,481,0
55+,0,0,0,0,0,0,372


In [54]:
df_join["age_group"] = df_join["age_group"].replace(ser_repl)

In [55]:
df_join.head(2)

,user,gender,age_group,job,city,marital,prod,purchase
0,1,0,0,10,A,0,35,334093
1,2,1,6,16,C,0,77,810472


In [57]:
# df_join_dum = pd.get_dummies(df_join, columns = ["job", "city"]) # 시험버전
df_join_dum = pd.get_dummies(df_join, columns = ["job", "city"], dtype = "int") # Pandas 2.0.0
df_join_dum = df_join_dum.drop(columns = "user")
df_join_dum.head(2)

,gender,age_group,marital,prod,purchase,job_0,job_1,job_2,job_3,job_4,...,job_14,job_15,job_16,job_17,job_18,job_19,job_20,city_A,city_B,city_C
0,0,0,0,35,334093,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,6,0,77,810472,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [58]:
arr_join_nor = MinMaxScaler().fit_transform(df_join_dum)
arr_join_nor[:1, ]

array([[0.        , 0.        , 0.        , 0.02843137, 0.02739807,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        ]])

In [ ]:
model_kmeans = KMeans(n_clusters = 7, random_state = 123)
model_kmeans.fit(arr_join_nor)

In [61]:
round(silhouette_score(arr_join_nor, labels = model_kmeans.labels_), 2)

0.18

In [62]:
import sklearn
sklearn.__version__

'1.2.2'